In [2]:
import torch
import torch.nn as nn

In [3]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
class multimaskattation(nn.Module):
    
    def __init__(self,d_in,d_out,dropout,num_heads,qkv_bias = False):
        super().__init__()
        self.d_out  = d_out
        self.q_weight = nn.linear(d_in,d_out,qkv_bias = False)
        self.k_weight = nn.linear(d_in,d_out,qkv_bias = False)
        self.v_weight = nn.linear(d_in,d_out,qkv_bias = False)
        self.dropout  = nn.Dropout(dropout)
        
    def forward(self,idx):
        batch,num_tokens,d_in = idx.shape
        q_matrix = self.q_weight(idx)
        k_matrix = self.k_weight(idx)
        v_matrix = self.v_weight(idx)
        
        attention_score = q_matrix @ k_matrix.T
        mask_metrix = torch.triu(torch.ones())
        masked_matrix = attention_score.masked_fill(mask_metrix.bool()[:num_tokens,:num_tokens],-torch.inf)
        attention_weight = nn.Softmax(masked_matrix / idx.shape[-1] ** 0.5)
        
        attention_weight = self.dropout(attention_weight)
        context_vec = attention_weight @ v_matrix
        return context_vec